In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
%matplotlib inline

In [5]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("Weather_Database/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kashi,CN,2022-10-26 00:09:22,39.4547,75.9797,49.41,33,100,3.00
1,1,Hithadhoo,MV,2022-10-26 00:09:22,-0.6000,73.0833,82.44,72,32,11.61
2,2,Flin Flon,CA,2022-10-26 00:07:55,54.7682,-101.8650,27.50,92,100,5.77
3,3,Mandal,IN,2022-10-26 00:09:22,25.4500,74.5667,69.58,36,0,5.14
4,4,Honningsvag,NO,2022-10-26 00:09:23,70.9821,25.9704,33.55,80,40,14.97


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Mandal,IN,2022-10-26 00:09:22,25.4500,74.5667,69.58,36,0,5.14
5,5,Mar Del Plata,AR,2022-10-26 00:08:17,-38.0023,-57.5575,67.08,85,100,14.00
10,10,Acari,PE,2022-10-26 00:09:24,-15.4311,-74.6158,66.13,73,9,5.28
11,11,Victoria,HK,2022-10-26 00:07:42,22.2855,114.1577,75.38,66,96,11.99
16,16,Pisco,PE,2022-10-26 00:09:26,-13.7000,-76.2167,60.85,82,0,18.41
18,18,Hamilton,US,2022-10-26 00:06:34,39.1834,-84.5333,73.80,57,20,3.44
20,20,Rikitea,PF,2022-10-26 00:09:27,-23.1203,-134.9692,73.42,74,62,16.28
21,21,Ponta Do Sol,PT,2022-10-26 00:09:27,32.6667,-17.1000,72.23,91,53,5.86
22,22,Touros,BR,2022-10-26 00:09:27,-5.1989,-35.4608,77.90,76,41,14.23
28,28,Adrar,MR,2022-10-26 00:09:29,20.5022,-10.0711,79.92,15,96,9.57


In [9]:
preferred_cities_df.count()

City_ID       258
City          258
Country       254
Date          258
Lat           258
Lng           258
Max Temp      258
Humidity      258
Cloudiness    258
Wind Speed    258
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Mandal,IN,69.58,25.4500,74.5667,
5,Mar Del Plata,AR,67.08,-38.0023,-57.5575,
10,Acari,PE,66.13,-15.4311,-74.6158,
11,Victoria,HK,75.38,22.2855,114.1577,
16,Pisco,PE,60.85,-13.7000,-76.2167,
18,Hamilton,US,73.80,39.1834,-84.5333,
20,Rikitea,PF,73.42,-23.1203,-134.9692,
21,Ponta Do Sol,PT,72.23,32.6667,-17.1000,
22,Touros,BR,77.90,-5.1989,-35.4608,
28,Adrar,MR,79.92,20.5022,-10.0711,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
 # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

 # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

 # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
 # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Mandal,IN,69.58,25.4500,74.5667,Surpanch saab ki vatika
5,Mar Del Plata,AR,67.08,-38.0023,-57.5575,Gran Hotel Mar del Plata
10,Acari,PE,66.13,-15.4311,-74.6158,HOSPEDAJE XIMENA
11,Victoria,HK,75.38,22.2855,114.1577,Mini Central
16,Pisco,PE,60.85,-13.7000,-76.2167,Hotel San Isidro Oficial
18,Hamilton,US,73.80,39.1834,-84.5333,Six Acres Bed & Breakfast
20,Rikitea,PF,73.42,-23.1203,-134.9692,People ThankYou
21,Ponta Do Sol,PT,72.23,32.6667,-17.1000,Hotel do Campo
22,Touros,BR,77.90,-5.1989,-35.4608,INN NEW HORIZON
28,Adrar,MR,79.92,20.5022,-10.0711,


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))